[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jmdelvecchio/ears33/blob/main/colab_data_tutorials/DT5_Probing_Adirondack_landslides_in_Google_Earth_Engine.ipynb)

# Introduction

Google Earth Engine is a powerful tool for Earth observation. There are nearly infinite possibilities for data analysis as the Google servers store petabytes of data and can perform many operations at lightning speed. However, the main vehicle for GEE (and most StackExchange resources, lol) is its JavaScript code editor, which is fine excxept I like Python and many folks like Python. So blessed people like [Qiusheng Wu](https://github.com/giswqs) at the University of Tennessee have written some Python packages to make GEE play nicely with our familiar Python syntax and plotting tools. 

Ensure you have registered for a Google Earth Engine account and have some landslide shapfiles in hand to upload to the JHub server. 

# Learning objectives:

- Use Google Earth Engine to explore multispectral datasets for the Adirondack area impacted by Irene
- Interpret trends in multispectral signals from the landscape over time as they are impacted by, and recover from, disturbance
- Understand the philosophy of [supervised classification](https://developers.google.com/earth-engine/guides/classification) of multispectral imagery and evaluate model results versus mapping by hand
- Judge the applicability of certain satellite data products for automated landscape disturbance detection

# A note for this data tutorial

I have written this data tutorial to be more like an interactive textbook chapter demonstrating ideas rather than having you working on your own. For that reason please read the background information and all links thoroughly - your deliverables will be more short answers and reflections than code outputs. 

# Background: NDVI and other spectral indices

Remember from lecture that [different land cover exhibits different intensities of visible and invisible wavelengths of energy](https://blog.descarteslabs.com/a-look-into-the-fundamentals-of-remote-sensing). Since certain land cover has characteristic high or low reflectances for cetain wavelengths (like green, red and infrared), folks have come up with spectral indices that discriminate between surface cover. One of the most used spectral indices is NDVI, or normalized difference vegetation index. Read about NDVI [here](https://gisgeography.com/ndvi-normalized-difference-vegetation-index/). A huge list of other indices, if you're curious, are [here](https://www.indexdatabase.de/db/i.php). 

# Install and import packages

In [ ]:
! pip install geemap --quiet

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt

import ee

import geemap # Qiusheng Wu rules!!!

import json

import os

# Authenticate and initialize GEE

If you haven't registered for GEE, this step will not work. 

Pay close attention to the messages that get displayed when you click through them. Google has made it more secure to get your authentication key but it means you'll get some scary looking messages. Just click "continue" or "yes" whenever it asks *"are you sure??"*


In [ ]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

# Define functions

You've seen functions before, but GEE is big on functions. GEE syntax is set up to leverage the `.map` function, which [maps an algorithm across an ImageCollection](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-map), or collection of satellite scenes that fit some criteria. I think their way is technically pretty Pythonic but it's counter-intuitive when you first think about it. 

The first few are straightforward - they just take an image from the ImageCollection and do something to every image (it's essentially saying `for i in enumerate(images), do something to images[i]`). 

The first function creates a [normalized spectral index](https://developers.google.com/earth-engine/apidocs/ee-image-normalizeddifference), in this case NDVI

In [ ]:
def NAIP_NDVI(image):
  index= image.normalizedDifference(['N','R']).rename('NDVI');
  return image.addBands(index)

In [ ]:
def clip_to_polygon(image):
  return(image).clip(polygon)

In [ ]:
def createTimeBand(image):
  return image.addBands(image.metadata('system:time_start').divide(3.154e10));

I wrote this ugly beast of a function below us in order to hav a somewhat modular tool to make an annual imge from a year's worth of images in an ImageCollection. It says "OK, how do we [reduce](https://developers.google.com/earth-engine/guides/reducers_image_collection) an entire set of images if we want to look at annual trends? Do we take a mean value of what we're intersted in? The max? And over which months?"

In [ ]:
def annual_images(y):
    range_year = ee.Filter.calendarRange(y, y, 'year')
    range_month = ee.Filter.calendarRange(start_month, end_month, 'month')
    filtered_dataset = (index_collection
                        .filter(range_year)
                        .filter(range_month)
                        .map(createTimeBand)) # Needed for linear regression 
    # Combine the mean and standard deviation reducers.
    if analysis == 'mean':
      reducers = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
      )
    elif analysis == 'min' or analysis == 'max':
      reducers = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.minMax(),
        sharedInputs=True
      )
    elif analysis == 'median':
      reducers = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.median(),
        sharedInputs=True
      )

# Use the combined reducer to get the mean and SD of the image.
    stats0 = filtered_dataset.reduce(
      reducer=reducers,
    )

    return stats0.set('year',y)

# adapted from https://gis.stackexchange.com/questions/392834/transform-google-earth-engine-script-to-python-with-landsat-8-temporal-data

# Demo: using pre-2009 landslide polygons

## Upload and load in shapefiles

Go to Files on the left and click Upload and select **ALL files** (not just `.shp` but `sbn`, `.prj` etc.) associated with your landslide shapefile. Here I'm showing you an example of the pre-2009 landslide shapefile that came with the lab. `

Don't forget if JHub is struggling to try `landslides = geemap.shp_to_ee('../../../pre2009.shp')`

In [ ]:
!wget https://github.com/jmdelvecchio/ears33/raw/main/colab_data_tutorials/pre_2009_slides.zip

!unzip /content/pre_2009_slides.zip

In [ ]:
landslides = geemap.shp_to_ee('./pre_2009_slides/pre2009.shp')

# Note that in addition to creating an Earth Engine feature collection, you have generated shapfiles that Earth Engine can understand, and these live in your "pre_2009_slides" directory
# We're just gonna have Colab print what "landslides" are. Check it out:
landslides 

I'm just going to define a polygon here to mimic the study area from the lab

In [ ]:
polygon = ee.Geometry.Polygon(
  [[[-73.83, 44.18],
   [-73.79, 44.15],
   [-73.87, 44.10],
   [-73.91, 44.13]]]
);

The `geemap.Map()` function creates an instance of an interactive map. If you want to start fresh every time you make a new map, you can call this function where you define your new map as `Map`. If you don't re-instantiate your map, you'll just update the previous `Map` instance. 

In [ ]:
Map = geemap.Map()
Map.setCenter(-73.849799,  44.137148, 12);
# The empty brackets can take arguments for min and max values for color display 
Map.addLayer(landslides, {}, 'Landslides')
Map.addLayer(polygon, {}, 'Polygon')
Map

## Load in an ImageCollection

[ImageCollections](https://developers.google.com/earth-engine/guides/ic_creating) are how Google stores its imagery and spectral data. They are "geocubes" in that they are spatial data with a number of bands over a number of collection times. You can [filter](https://developers.google.com/earth-engine/guides/ic_filtering) these ImageCollections by spatial or temporal bounds. 

This particular `ImageCollection` is the from the National Agriculture Imagery Program (NAIP), hosted on [Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ). Read about the product to learn the frequency with which the product is collected and the resolution at which it is collected. 

In [ ]:
NAIP = ee.ImageCollection('USDA/NAIP/DOQQ');

naip_2009 = NAIP.filter(ee.Filter.date('2009-01-01', '2009-12-31'))
naip_2013 = NAIP.filter(ee.Filter.date('2013-01-01', '2013-12-31'))
naip_2019 = NAIP.filter(ee.Filter.date('2019-01-01', '2019-12-31'))

In [ ]:
trueColorVis = {
  min: 0.0,
  max: 255.0,
}

Here I am selecting the red (R), green (G), and blue (B) bands to create a true color image. [Select](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-select) lets you choose a subset of an ImageCollection's data layers (most often bands). Read about `addLayer` [here](https://developers.google.com/earth-engine/apidocs/map-addlayer). 

On the resulting map, you can click the wrench icon to open up the interactive toolbar. Clicking on the "Layers" icon to the left of the wrench will open up a menu where you can use the slider to adjust the opacity of each layer. 

In [ ]:
Map = geemap.Map()
Map.setCenter(-73.849799,  44.137148, 12);
Map.addLayer(naip_2009.select(['R', 'G', 'B']), trueColorVis, '2009');
Map.addLayer(naip_2013.select(['R', 'G', 'B']), trueColorVis, '2013');
Map.addLayer(naip_2019.select(['R', 'G', 'B']), trueColorVis, '2019');
Map

## Add a spectral index to the ImageCollection

Here we will employ that `.map` function I mentioned earlier. I am going to filter the NAIP data by date starting in 2013 when they began collection of the infrared band (no infrared and therefore no NDVI before 2013).

In [ ]:
adk_ndvi_all= NAIP.filter(ee.Filter.date('2013-01-01', '2019-12-31')).map(NAIP_NDVI)

Here I am adding the year-specific NAIP collections and then selecting the NDVI band I created. 

In [ ]:
Map = geemap.Map()
Map.setCenter(-73.849799,  44.137148, 12);
Map.addLayer(naip_2019.map(NAIP_NDVI).select(['NDVI']), {}, '2019 NDVI');
Map.addLayer(naip_2013.map(NAIP_NDVI).select(['NDVI']), {}, '2013 NDVI');
Map

## Reduce spectral index data to an annual image

OK, so this is where you can use my state-of-the-art inefficient code to take any ImageCollection you want, select one of its bands, and reduce each year's worth of imagery to a single image per year. 

This code is a little mismatched with NAIP because NAIP imagery is collected only once over a location every three years. I originally wrote this script to look at MODIS data, which came as an average value every 16 days, so in the Arctic I wanted to know what they yearly maximum NDVI was over my field site per year. 

In [ ]:
# You can adjust these numbers, but I wouldn't recommend it to answer the questions
# in this data tutorial.
# If you get an error or no data, you won't find out here. 

# Options are 'mean', 'median', 'min' 'max'
analysis = 'max'

start_year=2013
end_year=2019
start_month=7
end_month=9
index_collection = adk_ndvi_all.select(['NDVI'])

# Don't make any adjustments below here 

years = ee.List.sequence(start_year,end_year)

yearwise_ndvi = years.map(annual_images)

# Make an ImageCollection from the list of images you just composited,
# since you need an ImageCollection for the linear fit reduction
yearCompCol = ee.ImageCollection.fromImages(yearwise_ndvi)

## Perform a pixelwise regression across the ImageCollection to obtain a trend in the spectral index over the observation period

Read about this function [here](https://developers.google.com/earth-engine/guides/reducers_regression). 

In [ ]:
# Get a pixelwise linear regression across the composited ImageCollection
# "select" is time and the band you are interested in
# The output is the slope of the line fit to each pixel's data over time
# and the timestep is "per year"
trend = yearCompCol.select(['system:time_start_mean', 'NDVI_' + analysis]).reduce(ee.Reducer.linearFit())

# 'system:time_start_mean' is my hacky way of doing time per scene
# The value is "the mean number of years since 1970 across the scene"
# which will just be the middle of the month(s) you chose in the year you chose

# The result is two outputs: "scale" is the slope and "offset" is the intercept

## Visualize results

Here are some ancillary topographic visualization collections for your `Map`!

In [ ]:
elevation = ee.Image("USGS/3DEP/10m").select('elevation');
hillshade = ee.Terrain.hillshade(elevation);
slope = ee.Terrain.slope(elevation);


In [ ]:
import geemap.colormaps as cm
palette = cm.palettes.ndvi

Map = geemap.Map()

Map.addLayer(slope, {'min': 0.0, 'max':60.0, 'palette': ['white', 'black']}, '3DEP Hillshade');
Map.addLayer(trend.select('scale'), {'min': -0.1, 'max':0.1, 'palette': palette}, 'NDVI trend')
Map.addLayer(naip_2009.select(['R', 'G', 'B']), trueColorVis, '2009');
Map.addLayer(naip_2013.select(['R', 'G', 'B']), trueColorVis, '2013');
Map.addLayer(naip_2019.select(['R', 'G', 'B']), trueColorVis, '2019');
Map.addLayer(landslides, {}, 'Landslides')
Map.setCenter(-73.849799,  44.137148, 12);
Map

See any new slides since 2013??

## Tabulate results for matplotlib-style plotting

I wrote this script for another application (spectral index trends in Arctic watersheds) but I figured I'd show this off in case you were curious. This script make a [histogram](https://developers.google.com/earth-engine/apidocs/ee-reducer-autohistogram) for each polygon (identified by `OBJECTID` or similar field in shapefile attribute table) and uses dictionaries and keys to pull that info out of the `autoHistogram` reducer in GEE. 

In [ ]:
lanslide_histos = trend.reduceRegions(collection=landslides,reducer=ee.Reducer.autoHistogram(maxBuckets=40), scale=1)

In [ ]:
info = lanslide_histos.getInfo()
hist_dict = {features['properties']['OBJECTID']:features['properties']['scale'] for features in info['features']}

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
for key in hist_dict:
  # First element in dictionary value is the bin edge
  bin_edges = [i[0] for i in hist_dict[key]]
  # Second element in dictionary value is counts in each bin
  bin_counts = [i[1] for i in hist_dict[key]]
  bin_counts_norm = bin_counts/np.max(bin_counts)
  ax.plot(bin_edges, bin_counts_norm, alpha=0.5)

#ax.set_xlim((-0.025, 0.025))
ax.set_xlabel("Linear trend of NDVI, " + str(start_year)+"-"+str(end_year))
ax.set_ylabel("Normalized frequency")
fig.suptitle("Distribution of Pixelwise Trends in NDVI for Each Landslide Scar")

# Machine learnin'

Did you feel like a computer when you were clicking out landslides from imagery? A good rule of thumb is that if you feel like a computer doing something, a computer *may* be able to do it (1) faster and or (2) better than you can! We can use the landslide polygons clicked out by hand to tell the computer "this is a landslide". Let's see how this works...

In [ ]:
Map = geemap.Map()
naip_clip = naip_2009.filterBounds(polygon).mean().clip(polygon)
Map.addLayer(naip_clip.select(['R', 'G', 'B']), trueColorVis, '2009 clipped')
# Map.addLayer(landslides, {}, 'Landslides')
Map.setCenter(-73.849799,  44.137148, 12);
Map

The following codeblock makes use of the functions [`reduceToImage`](https://developers.google.com/earth-engine/apidocs/ee-featurecollection-reducetoimage) and [`sample`](https://developers.google.com/earth-engine/apidocs/ee-image-sample). 

In [ ]:
# This code block makes a binary "yes landslides" or "no landslides" based on the shapefile
landslide_raster = landslides.reduceToImage(
    properties=['OBJECTID'],
    reducer=ee.Reducer.first().setOutputs(['occurrence'])
).gt(0)

binary_training_data = landslide_raster.unmask(0)
# "unmask" fills nodata with a value


# Make the training dataset.
random_points = binary_training_data.sample(
    **{
        'region': polygon,
        'scale': 10,
        'numPixels': 1000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

# Because the landslides are only a small portion of the area I am cheating and specifically collecting some points from landslides
# This is veering toward an "imbalanced" training dataset where the training data has more landslides than the real data have
slide_points = binary_training_data.sample(
    **{
        'region': landslides,
        'scale': 10,
        'numPixels': 100,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

# Now add the two sets of points together
points = random_points.merge(slide_points)



Now we say "for those points we just created, tell me about the values of the R, G, and B bands at those points." Which, if you think about it, is the same thing your eyeballs did for each pixel when you were cruising ArcMap, right?

In [ ]:
# Use these bands for prediction.
NAIP_bands = ['R', 'G', 'B']

# This property of the table stores the label of interest.
label = 'occurrence'

# Overlay the points on the imagery to get training.
training = naip_clip.select(NAIP_bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 1}
)

trainingNoNulls = training.filter(
  ee.Filter.notNull(training.first().propertyNames())
);


And now we [train](https://developers.google.com/earth-engine/apidocs/ee-classifier-train) our model. Read bout the classifier [here](https://developers.google.com/earth-engine/apidocs/ee-classifier-smilecart). 

In [ ]:
trained = ee.Classifier.smileCart().train(trainingNoNulls, label, NAIP_bands);


# Train a CART classifier with default parameters.
# trained = ee.Classifier.smileCart().train(training, label, NAIP_bands)



And now that we have a model, or a predicted relationship between three band values and properties about the image (in this case, whether or not the image is a landslide or not) for a few hundred pixels, we are going to [classify](https://developers.google.com/earth-engine/apidocs/ee-image-classify) an "unknown" dataset, in this case the rest of the image's pixels. 

In [ ]:
# Classify the image with the same bands used for training.
result = naip_clip.select(NAIP_bands).classify(trained)

# focal_mode makes the image less noisy by finding the most frequent classification value in a given radius
result=result.focal_mode()


Let's look at the result!

In [ ]:

Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

# Machine learnin': now your turn!

What if we tried to use the trained classifier on the 2013 imagery to find post-Irene landslides? Overlay your shapefiles in the mapping codeblock below to see how the model compares to your clicks. 

In [1]:
# Codebloack to bring your polygons into Earth Engine

# Be sure to plug your variables in correctly in the block below

In [ ]:
Map = geemap.Map()
naip_clip_13 = naip_2013.filterBounds(polygon).mean().clip(polygon)
Map.addLayer(naip_clip.select(['R', 'G', 'B']), trueColorVis, '2013 clipped')

result = naip_clip_13.select(NAIP_bands).classify(trained)

Map.addLayer(result.randomVisualizer(), {}, 'classified')

##### Your edits here
# Map.addLayer(???, {}, '??')
#####

Map.setCenter(-73.849799,  44.137148, 12);
Map

Finally, what if we tried to find brand-new landslides from the 2019 imagery?

In [ ]:
Map = geemap.Map()
naip_clip_19 = naip_2019.filterBounds(polygon).mean().clip(polygon)
Map.addLayer(naip_clip.select(['R', 'G', 'B']), trueColorVis, '2019 clipped')

result = naip_clip_19.select(NAIP_bands).classify(trained)

# result=result.focal_mode()

Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map.setCenter(-73.849799,  44.137148, 12);
Map

What do you think - are geomorphologists out of a job yet?

# Is that the best we can do?

Recall that in the first part of the notebook, we saw how good NDVI was at telling us where vegetation was missing and therefore where we had landslides. Then, also recall in the second half of the notebook, we were training a landslide classifier on *three* bands - reg, green, and blue - on the NAIP imagery. 

Using all the code you've seen in this notebook, can you train a new classifier on *all* the bands available from the NAIP dataset? Does it improve your predictions?

(Note that the near infrared bands are unavailable in 2009 so we're going to have to use 2013 imagery and thus use all of our landslide polygons)

## Step 1: load in b oth pre- and post-Irene landslides

In [ ]:
landslides_1 = gpd.read_file("pre2009.shp")
landslides_2 = gpd.read_file("post2009.shp") # What are your files called?

slides_combined = pd.concat([landslides_1, landslides_2]).reset_index().to_file("slides_combined.shp")

## Step 2: Create new training data points

(Nothng new to edit here)

In [ ]:
landslides = geemap.shp_to_ee("slides_combined.shp")

# This code block makes a binary "yes landslides" or "no landslides" based on the shapefile
landslide_raster = landslides.reduceToImage(
    properties=['OBJECTID'],
    reducer=ee.Reducer.first().setOutputs(['occurrence'])
).gt(0)

binary_training_data = landslide_raster.unmask(0)
# "unmask" fills nodata with a value


# Make the training dataset.
random_points = binary_training_data.sample(
    **{
        'region': polygon,
        'scale': 10,
        'numPixels': 1000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

# Because the landslides are only a small portion of the area I am cheating and specifically collecting some points from landslides
# This is veering toward an "imbalanced" training dataset where the training data has more landslides than the real data have
slide_points = binary_training_data.sample(
    **{
        'region': landslides,
        'scale': 10,
        'numPixels': 100,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

# Now add the two sets of points together
points = random_points.merge(slide_points)

## Step 3: select bands for training data and analysis and do the training and classification.

Don't forget you can view band names on [Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ). 

In [ ]:
# Use these bands for prediction.
NAIP_bands = ['R', 'G', 'B', '???']

# This property of the table stores the label of interest.
label = 'occurrence'

# Overlay the points on the imagery to get training.
training = naip_clip_13.select(NAIP_bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 1}
)

trainingNoNulls = training.filter(
  ee.Filter.notNull(training.first().propertyNames())
);

trained = ee.Classifier.smileCart().train(trainingNoNulls, label, NAIP_bands);

# Classify the image with the same bands used for training.
result = naip_clip_13.select(NAIP_bands).classify(trained)

# focal_mode makes the image less noisy by finding the most frequent classification value in a given radius
result=result.focal_mode()

Map.addLayer(naip_2019.select(['R', 'G', 'B']), trueColorVis, '2019');
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map.setCenter(-73.849799,  44.137148, 12);
Map

# Reflection questions


1.   Why might we use NDVI to track landslide activity? What benefits does this spectral index have over the RGB imagery you used in the lab? What are drawbacks? Did including extra bands improve your classifier?
2.   Functionally/in real life, what do the trends in NDVI of the landslide scars tell us about how the landscape is changing? Is there a difference in the NDVI trends between the pre- and post-Irene landslides? What do you think sets that pattern?
3. NAIP imagery has a resolution of 1 m and is collected once every three years for an area of the US. But there are other datasets out there that have visible and near-infrared bands like [MODIS](https://modis.gsfc.nasa.gov/data/dataprod/mod13.php) and [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED). List both the **spatial** and **temporal** resolution of each of these three instruments. What are the benefits and drawbacks of using each of these three datasets to detect landslides like we did in this activity? 


Your text here